In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.style.use("ggplot")
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 14
from scipy.integrate._ivp.ivp import OdeResult
from scipy.optimize import curve_fit
import seaborn as sns
import numpy as np
import tqdm, os

from ars_247_2025_final_project.models import TRAM_Model
from ars_247_2025_final_project.tools import gaussian_pulse, double_sigmoid_pulse

In [ ]:
OUTPUT_DIR = os.path.abspath(os.path.join("..", "Final_Project_PNG"))
print(f"Saving files to {OUTPUT_DIR}...")
HOUR = 3600
MINUTE = 60
# transcriptional pulse
transcriptional_pulse = {
    'peak_time': 1.5 * HOUR,
    'peak_value': 1,
    'width': 0.5 * HOUR
}

# phosphorylation pulse
phosphorylation_pulse = {
    'peak_time': 1.5 * HOUR // 4,
    'peak_value': 1,
    'width': 0.5 * HOUR // 4 # 45 mins peak phosphorylation pulse
}

# optogenetic pulse
optogenetic_pulse = {
    'peak_time': 1.5 * HOUR // 10, # peak at 9 mins
    'peak_value': 1,
    'width': 0.5 * HOUR // 10 # 3 mins peak activity of pulse
}

## Figure 1

In [ ]:
system_params = {
    'PROM': 1,
    'V_A': 0.47, #um/s - mean for dynein / kinesin
    'R_M': 2.5, #um - mean for mammalian cells is 2.5; best results w/ 20
    'K_TEV': 0.015, #/s - physiological is ~0.15
    'K_TVMV': 0.015, #/s - physiological is ~0.15
    'K_TRAM_DEG': 0.00001, #/s - got interesting results with this being 0.01; physiological is ~0.00001
    'K_TF_DEG': 0.01 #/s - got interesting results with this being 0.1; physiological is ~0.01 w/ fast degron
}


In [ ]:
THIS_PULSE = optogenetic_pulse
models: dict[int, TRAM_Model] = {}
solutions: dict[int, OdeResult] = {}
T = 24 * HOUR
for i in tqdm.tqdm(range(0, 9)):
    models[i] = TRAM_Model(
        n_delays = i, 
        pulse = 'gaussian', pulse_params = THIS_PULSE, 
        sys_config = system_params
    )
    solutions[i] = models[i].solve_delay_ivp(0, T)

### Validating Input Pulse

In [ ]:
t = np.arange(0, 3 * HOUR)
pulse = [gaussian_pulse(y, **THIS_PULSE) for y in t]
plt.plot(t, pulse)

In [ ]:
# Area under pulse curve
THIS_N = 8
pulse_soln = models[THIS_N].solve_pulse()
plt.plot(pulse_soln['t'], pulse_soln['y'][0, :], color='k', label='Input')
ax = plt.gca()
ax.set_xticks(np.arange(0,3*HOUR,HOUR // 2))
ax.set_xticklabels(ax.get_xticks() / HOUR)

### Fig 1a
Demonstrating the output curve given physiological parameters

Starting with 4 TRAM domains

In [ ]:
trans_model_4_tram = TRAM_Model(
        n_delays = 4, 
        pulse = 'gaussian', pulse_params = transcriptional_pulse, 
        sys_config = system_params
)
soln_trans_model_4_tram = trans_model_4_tram.solve_delay_ivp(0, T)

In [ ]:
THIS_N = 4
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Pulse Peak\n(Input)', color='g', ls='--')
for i in list(range(THIS_N * 4))[::4]:
    this_alpha = i / 4 / THIS_N + 1 / THIS_N
    this_label = "Trafficking\nIntermed." if i == THIS_N * (4 - 1) else None
    plt.plot(soln_trans_model_4_tram['t'], soln_trans_model_4_tram['y'][i, :], label=this_label, color='k', alpha=this_alpha)
plt.plot(soln_trans_model_4_tram['t'], soln_trans_model_4_tram['y'][-1, :], label='TF (Output)', color='m')
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 3.5 * HOUR)
ax.set_xticks(np.arange(0,3.5*HOUR,HOUR // 2))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.04, y=0.93, s="a",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=18, fontweight='bold', style='italic'
)
plt.legend(loc="upper right", handletextpad=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1a.png"), dpi=400)

### Fig 1b
Demonstrating the output curve given physiological parameters & optogenetic stimulation

Starting with 4 TRAM domains

In [ ]:
THIS_N = 4
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Pulse Peak (Input)', color='g', ls='--')
for i in list(range(THIS_N * 4))[::4]:
    this_alpha = i / 4 / THIS_N + 1 / THIS_N
    this_label = "Trafficking\nIntermediates" if i == THIS_N * (4 - 1) else None
    plt.plot(solutions[THIS_N]['t'], solutions[THIS_N]['y'][i, :], label=this_label, color='k', alpha=this_alpha)
plt.plot(solutions[THIS_N]['t'], solutions[THIS_N]['y'][-1, :], label='TF (Output)', color='m')
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 1 * HOUR)
ax.set_xticks(np.arange(0,1*HOUR,HOUR // 4))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.03, y=0.93, s="b",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=18, fontweight='bold', style='italic'
)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1b.png"), dpi=400)

### Fig 1c
Determining the relationship between TRAM domains and output peak time

In [ ]:
pulse_soln = models[THIS_N].solve_pulse()
plt.axvline(THIS_PULSE['peak_time'], label='Input', color='g', ls='--')
for n_tram_domains, soln in solutions.items():
    this_alpha = n_tram_domains / len(models) + 1 / len(models)
    plt.plot(soln['t'], soln['y'][-1,:], label=f"{n_tram_domains}xTRAM", color="m", alpha=this_alpha)
plt.ylabel("REU")
ax = plt.gca()
plt.xlim(0, 1 * HOUR)
ax.set_xticks(np.arange(0,1*HOUR,HOUR // 4))
ax.set_xticklabels(ax.get_xticks() / HOUR)
plt.xlabel("Time (hr)")
plt.figtext(
    x=0.045, y=0.92, s="c",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=18, fontweight='bold', style='italic'
)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1c.png"), dpi=400)

### Fig 1d
Determining relationship between delay (t) and n tram domains (n)

In [ ]:
pulse_soln = models[THIS_N].solve_pulse()
input_peak = THIS_PULSE['peak_time']
delays = np.zeros(shape=(len(solutions),))
for sdx, soln in solutions.items():
    mdx = np.argmax(soln['y'][-1,:])
    t = soln['t'][mdx]
    delays[sdx] = t - input_peak
x = np.array(list(solutions.keys()))
plt.scatter(x, delays, color='k')
ax = plt.gca()
print(ax.get_ylim()[1])
max_y_val = np.ceil(ax.get_ylim()[1] / MINUTE) * MINUTE
print(max_y_val)
ax.set_ylim(0, max_y_val)
ax.set_yticks(np.arange(0, max_y_val, 2 * MINUTE))
ax.set_yticklabels(ax.get_yticks() / MINUTE)
plt.xlabel('NxTRAM Domains')
plt.ylabel('Delay (min)')

# Fitting Curve
delay_m, delay_b = np.polyfit(x, delays, 1)
delay_line = np.poly1d([delay_m, delay_b])
delay_line_r_squared = 1 - \
    (np.sum((delays - delay_line(x))**2) / \
     np.sum((delays - np.mean(delays)**2)))
plt.plot(x, delay_line(x), color='g')
plt.legend(
    [
        "Peak Output Time Delay",
        f"{delay_m:.2f}N + {delay_b:.2f} [R\u00B2={delay_line_r_squared:.2f}]"
    ],
    # loc="lower center",
    # bbox_to_anchor=(0.45, -0.45),
    # ncol = 2
)
plt.figtext(
    x=0.03, y=0.93, s="d",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=18, fontweight='bold', style='italic'
)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1d.png"), dpi=400)

### Fig 1e
Determining relationship between delay (t) and n tram domains (n)

In [ ]:
# Graphing points
pulse_soln = models[THIS_N].solve_pulse()
input_peak = THIS_PULSE['peak_time']
peaks = np.zeros(shape=(len(solutions),))
for sdx, soln in solutions.items():
    peaks[sdx] = np.max(soln['y'][-1,:])
x = np.array(list(solutions.keys()))
plt.scatter(x, peaks, color='k')
ax = plt.gca()
ax.set_ylim(0, ax.get_ylim()[1])
plt.xlabel('NxTRAM Domains')
plt.ylabel('Peak Output (REU)')

# Fitting Curve
def log_fit(x, a, o, b, c):
    return - a * np.log(x + o) / np.log(b) + c

initial_guesses = [1, 1, np.e, 60]
popt, pcov = curve_fit(log_fit, x, peaks, p0 = initial_guesses)
a_fit, o_fit, b_fit, c_fit = popt

plt.plot(x, log_fit(x, a_fit, o_fit, b_fit, c_fit), color='g')
peak_line_r_squared = 1 - \
    (np.sum((peaks - log_fit(x, a_fit, o_fit, b_fit, c_fit))**2) / \
     np.sum((peaks - np.mean(peaks)**2)))
plt.legend(
    [
        "Peak Output",
        f"-{a_fit:.1f} \u2022 log{b_fit:.1f}(x + {o_fit:.1f}) + {c_fit:.1f} [R\u00B2={peak_line_r_squared:.2f}]"
    ], 
    # loc="lower center",
    # bbox_to_anchor=(0.45, -0.45),
    # ncol = 1
)
plt.figtext(
    x=0.04, y=0.93, s="e",
    horizontalalignment='left',
    verticalalignment='center',
    fontsize=18, fontweight='bold', style='italic'
)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "Fig1e.png"), dpi=400)